# PyPremise Example:  LLM outputs

This notebook demonstrates how to use PyPremise to analyze how different prompts lead to different outputs from a language model (in this case, OpenAI's GPT-4o-mini).

We generate outputs for two prompts, label them, and then use PyPremise to find patterns that distinguish between them.

We begin by installing the OpenAI python package

In [ ]:
pip install openai

Import all the packages we will need.

In [1]:
from openai import OpenAI
from pypremise import Premise, data_loaders
from tqdm import tqdm
import json

To use GPT-4o-mini, you need an API key. If you don’t have one yet, you can get it from https://platform.openai.com/.

Security Tip: Never share your API key or upload it to public code repositories.

In [ ]:
# Replace with your own OpenAI API key
key = "sk-..."  # <-- Make sure to keep this secret
client = OpenAI(api_key=key)


We define a helper function `llm_call(prompt, seed)` to call GPT-4o-mini.

The `seed` is important here: it ensures **reproducibility**, meaning that if you use the same prompt and seed, you’ll get the same output every time.

In [3]:
def llm_call(prompt, seed):
    """
    Calls GPT-4o with a specific prompt and seed.
    Returns the generated text from the model.
    """
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        seed=seed,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content


This is the core function where the experiment happens. Here's the logic:

1. Call GPT-4o-mini multiple times with each prompt

2. Tokenize the outputs into word lists

3. Label them (0 = from prompt1, 1 = from prompt2)

4. Use PyPremise to find patterns that differentiate the two classes

In [ ]:
def run_experiment(experiment_name, prompt1, prompt2, repetitions):
    """
    Calls the model with two prompts multiple times, labels the results,
    and uses PyPremise to find patterns that distinguish them.
    """
    # Generate text instances from both prompts
    instances1 = [llm_call(prompt1, i).split(" ") for i in tqdm(range(repetitions), desc="Prompt 1")]
    instances2 = [llm_call(prompt2, i).split(" ") for i in tqdm(range(repetitions), desc="Prompt 2")]
    instances = instances1 + instances2
    labels = [0]*repetitions + [1]*repetitions

    # Convert to PyPremise input format
    premise_instances, _, voc_index_to_token = data_loaders.from_token_lists_and_labels(instances, labels)
    premise = Premise(voc_index_to_token=voc_index_to_token)

    # Discover patterns
    patterns = premise.find_patterns(premise_instances)

    # Save results
    results = {
        "b_prompt1": prompt1,
        "b_prompt2": prompt2,
        "instances1": instances1,
        "instances2": instances2,
        "a_patterns": [str(p) for p in patterns]
    }

    with open("result_" + experiment_name + ".json", "w") as out_file:
        json.dump(results, out_file, sort_keys=True, indent=4)

    # Also return patterns for inspection
    return patterns


Let’s say we want to understand how GPT-4o-mini talks differently about doctors vs. teachers.

We define two prompts that only differ in the profession mentioned.

In [5]:
prompt1 = "Tell a 50 word story about a doctor."
prompt2 = "Tell a 50 word story about a teacher."

Let’s generate 30 examples from each prompt (for faster testing).

You can increase this number later to improve pattern quality.

In [8]:
patterns = run_experiment("doctor_vs_teacher", prompt1, prompt2, repetitions=30)

Prompt 2: 100%|██████████| 30/30 [00:56<00:00,  1.89s/it]


Let’s now look at the top patterns that PyPremise found.
Each pattern is a word (or phrase) that strongly correlates with either the doctor or teacher prompt.

In [9]:
for i, pattern in enumerate(patterns):
    print(f"Pattern {i+1}: {pattern}")

Pattern 1: (Dr.) towards group 0 (Instances: 30 in group 0, 0 in group 1)
Pattern 2: (stood) and (before) towards group 1 (Instances: 0 in group 0, 17 in group 1)
Pattern 3: (rushed) towards group 0 (Instances: 21 in group 0, 0 in group 1)
Pattern 4: (Thompson) towards group 1 (Instances: 0 in group 0, 20 in group 1)
Pattern 5: (A) towards group 0 (Instances: 20 in group 0, 1 in group 1)
Pattern 6: (through) towards group 0 (Instances: 23 in group 0, 3 in group 1)
